# 資料前處理 & 情緒分析


套件設定

In [277]:
import jieba
import jieba.analyse
import pandas as pd
%matplotlib inline

## MetaData
- 資料取得方式 : PTT 爬蟲
- 資料來源 : 汽車版、汽車買賣版
- 資料區間：2020/12/01 ~ 2023/01/31

| 廠牌 | 爬蟲關鍵字 | 資料總筆數 | 清理後筆數 |
| --- | --- | --- | --- |
| Nissan | Nissan、裕隆、裕日,日產、Sentra、Kicks、仙草 | 2,464 | 2,079 |
| Toyota | Toyota、Altis、Cross、豐田、和泰、阿提斯、卡羅拉 | 8,755 | 7,420 |
| Ford | 福特、六和、九和、上正、ford、Focus | 4,684 | 4,011 |
| Honda | honda、HRV、本田 | 2,488 | 2,171 |
| Mazda | mazda、CX-3、CX-30、馬三、mazda 3 | 3,321 | 2,787 |

`清理之定義為清除有空值的資料`

## 資料載入
根據自己的data路徑修改 <br>
`data不上github`，要注意ignore

In [278]:
ptt = pd.read_csv("../data/rawData/mazda_ptt_data.csv") 
ptt.head(3)

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01 00:09:42,city0504,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""mingchaoliu""...",111.243.121.95,2020-12-01 00:04:07,ptt
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 11:55:33,jerrysuper,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""XXXXBANG"", ""...",111.251.90.133,2020-12-02 00:04:03,ptt
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 21:53:41,twnail,car,cc一個月掛牌5319台\n國瑞產能這麼高丫?\n可以一天生產177台cc\n\n不知道它廠...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""luke2007"", ""...",42.72.60.253,2020-12-02 00:04:08,ptt


In [279]:
# 看看有幾篇文章
print(f"number of posts: {ptt.shape[0]}")
print(f"date range: {(ptt['artDate'].min(), ptt['artDate'].max())}")
print(f"category: \n{ptt['artCatagory'].value_counts()}")


number of posts: 2488
date range: ('2020-12-01 00:09:42', '2023-01-30 23:14:50')
category: 
artCatagory
car        1396
CarShop    1092
Name: count, dtype: int64


### 留言萃取
取出`artComment`的`cmtContent`

In [280]:
ptt = ptt[ptt.artComment != '[]'] # 刪除沒有comment的文章

# 取出 commentContent
def getComtInfo(com):
    cmtContent = ""
    com = eval(com)
    # print(com)
    for i in com:
        # print(i)
        cmtContent += i['cmtContent'] + "。"
    return pd.Series([cmtContent])


ptt[['cmtContent']] = ptt['artComment'].apply(lambda r: getComtInfo(r))
ptt.head(3)

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,cmtContent
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01 00:09:42,city0504,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""mingchaoliu""...",111.243.121.95,2020-12-01 00:04:07,ptt,:要看樓主住哪或開車活動範圍在哪？西部是還好啦。:一年是可以去保養廠幾次。:3008快小改款...
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 11:55:33,jerrysuper,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""XXXXBANG"", ""...",111.251.90.133,2020-12-02 00:04:03,ptt,:反指標板真的狂風向反著吹。:Kuga贏CRV。:雨衣廠商連帶受惠。:看車版以為RAV4銷量...
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 21:53:41,twnail,car,cc一個月掛牌5319台\n國瑞產能這麼高丫?\n可以一天生產177台cc\n\n不知道它廠...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""luke2007"", ""...",42.72.60.253,2020-12-02 00:04:08,ptt,:福特在中壢。:國瑞還有一個觀音廠可以上官網看一下。:我猜福特產能大概就月3000-4000...


將 `artTitle`, `artContent`, `cmtContent` 合併成新欄位 `whole_content`

In [281]:
ptt['whole_content'] = ptt['artTitle'] + ptt['artContent'] + ptt['cmtContent']
ptt = ptt[['system_id', 'artUrl', 'artTitle', 'artDate', 'artCatagory', 'artContent', 'whole_content']]
ptt

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01 00:09:42,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH原文連結：\nhttps://...
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 11:55:33,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,[情報]2020年11月份臺灣汽車市場銷售報告新增小七車\nhttps://www.7car...
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 21:53:41,car,cc一個月掛牌5319台\n國瑞產能這麼高丫?\n可以一天生產177台cc\n\n不知道它廠...,Re:[情報]2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台\n國瑞產能這麼...
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01 23:41:16,car,11月新車銷量 (車種銷量要等6號)\nhttps://response.jp/artic...,"[情報]日本到11月為止,各家新車累計銷量11月新車銷量 (車種銷量要等6號)\nhttp..."
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02 01:22:51,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,Re:[情報]2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 500...
...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27 19:12:28,CarShop,徵求菜單：HONDA HRV S+ 勁化版\n\n車輛狀況：全新 和 全新領牌車都可 請各自...,[購車]2023HONDAHRVS+勁化版徵求菜單：HONDA HRV S+ 勁化版\n\n...
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30 16:09:42,CarShop,車輛狀況：2023年 全新\n\n車輛品牌：HONDA\n\n車款型式：CR-V Vti-s...,[購車]HondaCR-VVti-s版/S版車輛狀況：2023年 全新\n\n車輛品牌：HO...
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30 19:10:38,CarShop,車輛狀況：全新\n\n車輛品牌：Honda\n\n車款型式：CRV Vti\n\n車輛顏色：...,[購車]HondaCRVVTI銀色車輛狀況：全新\n\n車輛品牌：Honda\n\n車款型式...
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30 21:14:28,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全\n\n車輛品牌：不拘，以非進口車為優先\...,[購車]5萬元以內代步車車輛狀況：(二手)無待修或是待修項目不影響行車安全\n\n車輛品牌：...


`artDate` 日期格式轉換

In [282]:
ptt["artDate"] = pd.to_datetime(ptt["artDate"])
ptt["artDate"] = ptt["artDate"].dt.date
ptt

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH原文連結：\nhttps://...
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,[情報]2020年11月份臺灣汽車市場銷售報告新增小七車\nhttps://www.7car...
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台\n國瑞產能這麼高丫?\n可以一天生產177台cc\n\n不知道它廠...,Re:[情報]2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台\n國瑞產能這麼...
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)\nhttps://response.jp/artic...,"[情報]日本到11月為止,各家新車累計銷量11月新車銷量 (車種銷量要等6號)\nhttp..."
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,Re:[情報]2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 500...
...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版\n\n車輛狀況：全新 和 全新領牌車都可 請各自...,[購車]2023HONDAHRVS+勁化版徵求菜單：HONDA HRV S+ 勁化版\n\n...
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新\n\n車輛品牌：HONDA\n\n車款型式：CR-V Vti-s...,[購車]HondaCR-VVti-s版/S版車輛狀況：2023年 全新\n\n車輛品牌：HO...
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新\n\n車輛品牌：Honda\n\n車款型式：CRV Vti\n\n車輛顏色：...,[購車]HondaCRVVTI銀色車輛狀況：全新\n\n車輛品牌：Honda\n\n車款型式...
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全\n\n車輛品牌：不拘，以非進口車為優先\...,[購車]5萬元以內代步車車輛狀況：(二手)無待修或是待修項目不影響行車安全\n\n車輛品牌：...


### 資料清理

In [283]:
# 清除空值
ptt.dropna(subset=['whole_content'], axis=0, how='any', inplace=True)

# 用'。'取代'\n\n'，並移除'\n'
ptt['whole_content'] = ptt['whole_content'].replace(r'[\n]+', '|', regex=True)
ptt = ptt.replace(r'[\n]+', '', regex=True)


# 移除內文中的網址
ptt['whole_content'] = ptt['whole_content'].str.replace('(http|https)://.*', '|', regex=True).replace(r'www\S+', '|', regex=True)

ptt

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：https://ctee.com.tw/lohas/car/378518.html...,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH原文連結：||
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車https://www.7car.tw/articles/read/70876?馬...,[情報]2020年11月份臺灣汽車市場銷售報告新增小七車||
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台國瑞產能這麼高丫?可以一天生產177台cc不知道它廠產能如何呢?像c...,Re:[情報]2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台|國瑞產能這麼高...
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)https://response.jp/article...,"[情報]日本到11月為止,各家新車累計銷量11月新車銷量 (車種銷量要等6號)||"
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,Re:[情報]2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 500...
...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版車輛狀況：全新 和 全新領牌車都可 請各自報價跟註...,[購車]2023HONDAHRVS+勁化版徵求菜單：HONDA HRV S+ 勁化版|車輛狀...
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新車輛品牌：HONDA車款型式：CR-V Vti-s版及S版車輛顏色...,[購車]HondaCR-VVti-s版/S版車輛狀況：2023年 全新|車輛品牌：HONDA...
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新車輛品牌：Honda車款型式：CRV Vti車輛顏色：銀色交易地區：不拘聯絡方...,[購車]HondaCRVVTI銀色車輛狀況：全新|車輛品牌：Honda|車款型式：CRV V...
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全車輛品牌：不拘，以非進口車為優先車款型式：...,[購車]5萬元以內代步車車輛狀況：(二手)無待修或是待修項目不影響行車安全|車輛品牌：不拘，...


### 替代字串

In [284]:
replace = pd.read_csv('../dict/replace.csv')
replace_dict = {key: ' ' for key in replace['alias']}

In [285]:
replace_dict

{'有無調表/實際里程': ' ',
 '欲售售價': ' ',
 'https?:\\/\\/(www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b([-a-zA-Z0-9()@:%_\\+.~#?&//=]*)': ' ',
 '車輛品牌/產地/出廠年月': ' ',
 '車款型式/排氣量': ' ',
 '原使用情形': ' ',
 '交易地區/聯絡方式': ' ',
 '實際照片': ' ',
 '其它說明': ' ',
 '外觀照': ' ',
 '本車缺點': ' ',
 '待修項目': ' ',
 '其他更換': ' ',
 '保養及維修紀錄': ' ',
 '車輛廠牌/年份/型號': ' ',
 '車輛原廠編成等級': ' ',
 '是否含保險': ' ',
 '是否含領牌稅金': ' ',
 '成交價格(總價)': ' ',
 '空車價格': ' ',
 '下訂日期範圍': ' ',
 '尋車地點': ' ',
 '保險類別/金額': ' ',
 '自費配備/配件': ' ',
 '贈送配備/配件': ' ',
 '付款方式(現金 or貸款)': ' ',
 '付款方式': ' ',
 '自費規費總額': ' ',
 '購車心得': ' ',
 '原文連結': ' ',
 '原文內容': ' ',
 '心得/說明': ' ',
 '(30字以上)': ' ',
 '心得或意見': ' ',
 '原文及網址連結': ' ',
 '綜合報導': ' ',
 '以下為建議資訊': ' ',
 '徵求菜單': ' ',
 '車款型式': ' ',
 '交易地區': ' ',
 '聯絡方式': ' ',
 '徵求業務': ' ',
 '購車地點': ' ',
 '車輛顏色': ' ',
 '車輛狀況': ' ',
 '交車地點': ' ',
 'B[0-9]+': ' ',
 '圖片來源': ' ',
 'Re:': ' ',
 'Fw:': ' ',
 '[新聞]': ' ',
 '[情報]': ' ',
 '[購車]': ' ',
 '[售車]': ' ',
 '[周邊]': ' ',
 '[菜單]': ' ',
 '[問題]': ' ',
 '[討論]': ' ',
 '[轉錄]'

In [286]:
def replace_str(data):
    for old, new in replace_dict.items():
        data = data.replace(old, "|")
    return data

In [287]:
replace_df = ptt.copy()
replace_df['whole_content'] = replace_df['whole_content'].apply(lambda x : replace_str(x))
replace_df

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：https://ctee.com.tw/lohas/car/378518.html...,|小休旅熱鬧好玩PEUGEOT30081.5LBlueH|：||
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車https://www.7car.tw/articles/read/70876?馬...,|2020年11月份臺灣汽車市場銷售報告新增小七車||
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台國瑞產能這麼高丫?可以一天生產177台cc不知道它廠產能如何呢?像c...,||2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台|國瑞產能這麼高丫?|可以...
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)https://response.jp/article...,"|日本到11月為止,各家新車累計銷量11月新車銷量 (車種銷量要等6號)||"
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,||2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 5000 輛是真...
...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版車輛狀況：全新 和 全新領牌車都可 請各自報價跟註...,|2023HONDAHRVS+勁化版|：HONDA HRV S+ 勁化版||：全新 和 全新...
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新車輛品牌：HONDA車款型式：CR-V Vti-s版及S版車輛顏色...,|HondaCR-VVti-s版/S版|：2023年 全新|車輛品牌：HONDA||：CR-...
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新車輛品牌：Honda車款型式：CRV Vti車輛顏色：銀色交易地區：不拘聯絡方...,|HondaCRVVTI銀色|：全新|車輛品牌：Honda||：CRV Vti||：銀色||...
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全車輛品牌：不拘，以非進口車為優先車款型式：...,|5萬元以內代步車|：(二手)無待修或是|不影響行車安全|車輛品牌：不拘，以非進口車為優先|...


### 斷詞
初始化斷詞引擎

In [288]:
jieba.set_dictionary('../dict/dict.txt')
jieba.load_userdict('../dict/user_dict.txt')

Building prefix dict from d:\Projects\NSYSU\2023_BigDataAnalysis\dict\dict.txt ...
Loading model from cache C:\Users\s2568\AppData\Local\Temp\jieba.uaa528441c6063f69433245c0db13322d.cache
Loading model cost 0.718 seconds.
Prefix dict has been built successfully.


先清除標點符號及空字串

In [289]:
clear_df = replace_df.copy()

clear_df['whole_content'] = clear_df['whole_content'].str.replace(r'[^\w\s\d]+', ' ', regex=True).astype(str)

clear_df

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：https://ctee.com.tw/lohas/car/378518.html...,小休旅熱鬧好玩PEUGEOT30081 5LBlueH
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車https://www.7car.tw/articles/read/70876?馬...,2020年11月份臺灣汽車市場銷售報告新增小七車
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台國瑞產能這麼高丫?可以一天生產177台cc不知道它廠產能如何呢?像c...,2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台 國瑞產能這麼高丫 可以一天...
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)https://response.jp/article...,日本到11月為止 各家新車累計銷量11月新車銷量 車種銷量要等6號
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 5000 輛是真的...
...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版車輛狀況：全新 和 全新領牌車都可 請各自報價跟註...,2023HONDAHRVS 勁化版 HONDA HRV S 勁化版 全新 和 全新領牌車...
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新車輛品牌：HONDA車款型式：CR-V Vti-s版及S版車輛顏色...,HondaCR VVti s版 S版 2023年 全新 車輛品牌 HONDA CR V V...
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新車輛品牌：Honda車款型式：CRV Vti車輛顏色：銀色交易地區：不拘聯絡方...,HondaCRVVTI銀色 全新 車輛品牌 Honda CRV Vti 銀色 不拘 來信附...
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全車輛品牌：不拘，以非進口車為優先車款型式：...,5萬元以內代步車 二手 無待修或是 不影響行車安全 車輛品牌 不拘 以非進口車為優先 4門...


進行jieba斷詞

In [290]:
# 設定繁體中文詞庫
jieba.set_dictionary("../dict/dict.txt.big")

stopwords_manual = ['恭喜', '有無', '有人', '是不是', '本來', '遇到', '機車', '時間', '討論', '10', 'XD', '20', '未來', '現在', '今年', '垃圾',
                    '11', '12', '30', '80', '15', '50', '不到', '2022', '2021', '2020', '最大', '不想', '樓上', '只能', '一次', '以前', '車的',
                    'KM', '公里', '正常']

# 新增stopwords
with open("../dict/stopwords.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]
stopWords.extend(stopwords_manual)

# 設定斷詞 function
def getToken(row):
    if not isinstance(row, str):  # 檢查類型是否為字串
        row = str(row)  # 將非字串類型轉換為字串
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

clear_df["words"] = clear_df["whole_content"].apply(getToken)
clear_df

Building prefix dict from d:\Projects\NSYSU\2023_BigDataAnalysis\dict\dict.txt.big ...
Loading model from cache C:\Users\s2568\AppData\Local\Temp\jieba.u87526c01a2c6093fa84ac3f5467b7506.cache


Loading model cost 1.302 seconds.
Prefix dict has been built successfully.


,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content,words
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：https://ctee.com.tw/lohas/car/378518.html...,小休旅熱鬧好玩PEUGEOT30081 5LBlueH,"[小休, 熱鬧, 好玩, PEUGEOT30081, 5LBlueH]"
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車https://www.7car.tw/articles/read/70876?馬...,2020年11月份臺灣汽車市場銷售報告新增小七車,"[月份, 臺灣汽車, 市場, 銷售, 報告, 新增, 小七車]"
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台國瑞產能這麼高丫?可以一天生產177台cc不知道它廠產能如何呢?像c...,2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台 國瑞產能這麼高丫 可以一天...,"[月份, 臺灣汽車, 市場, 銷售, 報告, cc, 一個月, 掛牌, 5319, 國瑞, ..."
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)https://response.jp/article...,日本到11月為止 各家新車累計銷量11月新車銷量 車種銷量要等6號,"[日本, 各家, 新車, 累計, 銷量, 新車, 銷量, 銷量]"
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 5000 輛是真的...,"[月份, 臺灣汽車, 市場, 銷售, 報告, Corolla, Cross, 5000, 輛..."
...,...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版車輛狀況：全新 和 全新領牌車都可 請各自報價跟註...,2023HONDAHRVS 勁化版 HONDA HRV S 勁化版 全新 和 全新領牌車...,"[2023HONDAHRVS, 勁化版, HONDA, HRV, 勁化版, 領牌車, 註明,..."
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新車輛品牌：HONDA車款型式：CR-V Vti-s版及S版車輛顏色...,HondaCR VVti s版 S版 2023年 全新 車輛品牌 HONDA CR V V...,"[HondaCR, VVti, 2023, HONDA, CR, Vti, 版及, 北部, ..."
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新車輛品牌：Honda車款型式：CRV Vti車輛顏色：銀色交易地區：不拘聯絡方...,HondaCRVVTI銀色 全新 車輛品牌 Honda CRV Vti 銀色 不拘 來信附...,"[HondaCRVVTI, 銀色, Honda, CRV, Vti, 銀色, 來信, 附上,..."
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全車輛品牌：不拘，以非進口車為優先車款型式：...,5萬元以內代步車 二手 無待修或是 不影響行車安全 車輛品牌 不拘 以非進口車為優先 4門...,"[以內, 代步車, 無待修, 影響, 行車, 安全, 以非, 進口車, 優先, 門或, 大眾..."


## 情緒分析
利用`LIWC`進行情緒分析
+ sentiment 計算方式: positive - anger - anx - negative - sad

In [291]:
senti_df = clear_df.copy()

In [292]:
# 讀取情緒字典
liwc_dict = pd.read_csv("../dict/liwc/LIWC_CH.csv")
liwc_dict = liwc_dict.rename(columns={'name': 'word', "class": 'sentiments'})
liwc_dict = liwc_dict.set_index('word')['sentiments'].to_dict()
# liwc_dict

In [293]:
def get_sentiment(words, liwc_dict):
    sentiment_ratio = 0
    pos = 0
    neg = 0
    for word in words:
        if word in liwc_dict:
            if (liwc_dict[word] == "positive"):
                pos += 1
            elif (liwc_dict[word] == "negative"):
                neg += 1
        else:
            continue
    
    if (pos+neg == 0):
        sentiment_ratio = 0.5
    else :
        sentiment_ratio = round(pos / (pos + neg), 3)
        
    return sentiment_ratio

In [294]:
# 幫每句話加上情緒分數
senti_df['sentimentRatio'] = senti_df.apply(lambda row : get_sentiment(row['words'],liwc_dict), axis = 1)
senti_df

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,whole_content,words,sentimentRatio
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：https://ctee.com.tw/lohas/car/378518.html...,小休旅熱鬧好玩PEUGEOT30081 5LBlueH,"[小休, 熱鬧, 好玩, PEUGEOT30081, 5LBlueH]",1.0
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車https://www.7car.tw/articles/read/70876?馬...,2020年11月份臺灣汽車市場銷售報告新增小七車,"[月份, 臺灣汽車, 市場, 銷售, 報告, 新增, 小七車]",0.5
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台國瑞產能這麼高丫?可以一天生產177台cc不知道它廠產能如何呢?像c...,2020年11月份臺灣汽車市場銷售報告cc一個月掛牌5319台 國瑞產能這麼高丫 可以一天...,"[月份, 臺灣汽車, 市場, 銷售, 報告, cc, 一個月, 掛牌, 5319, 國瑞, ...",0.5
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)https://response.jp/article...,日本到11月為止 各家新車累計銷量11月新車銷量 車種銷量要等6號,"[日本, 各家, 新車, 累計, 銷量, 新車, 銷量, 銷量]",0.5
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,2020年11月份臺灣汽車市場銷售報告Corolla Cross能賣破 5000 輛是真的...,"[月份, 臺灣汽車, 市場, 銷售, 報告, Corolla, Cross, 5000, 輛...",0.5
...,...,...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版車輛狀況：全新 和 全新領牌車都可 請各自報價跟註...,2023HONDAHRVS 勁化版 HONDA HRV S 勁化版 全新 和 全新領牌車...,"[2023HONDAHRVS, 勁化版, HONDA, HRV, 勁化版, 領牌車, 註明,...",1.0
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新車輛品牌：HONDA車款型式：CR-V Vti-s版及S版車輛顏色...,HondaCR VVti s版 S版 2023年 全新 車輛品牌 HONDA CR V V...,"[HondaCR, VVti, 2023, HONDA, CR, Vti, 版及, 北部, ...",0.5
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新車輛品牌：Honda車款型式：CRV Vti車輛顏色：銀色交易地區：不拘聯絡方...,HondaCRVVTI銀色 全新 車輛品牌 Honda CRV Vti 銀色 不拘 來信附...,"[HondaCRVVTI, 銀色, Honda, CRV, Vti, 銀色, 來信, 附上,...",1.0
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全車輛品牌：不拘，以非進口車為優先車款型式：...,5萬元以內代步車 二手 無待修或是 不影響行車安全 車輛品牌 不拘 以非進口車為優先 4門...,"[以內, 代步車, 無待修, 影響, 行車, 安全, 以非, 進口車, 優先, 門或, 大眾...",0.6


In [295]:
# 看一下情緒的分布
senti_df['sentimentRatio'].describe()

count    2171.000000
mean        0.613326
std         0.246357
min         0.000000
25%         0.500000
50%         0.500000
75%         0.800000
max         1.000000
Name: sentimentRatio, dtype: float64

In [296]:
final_df = senti_df.drop('whole_content', axis=1)
final_df

,system_id,artUrl,artTitle,artDate,artCatagory,artContent,words,sentimentRatio
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01,car,原文連結：https://ctee.com.tw/lohas/car/378518.html...,"[小休, 熱鬧, 好玩, PEUGEOT30081, 5LBlueH]",1.0
1,2,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,新增小七車https://www.7car.tw/articles/read/70876?馬...,"[月份, 臺灣汽車, 市場, 銷售, 報告, 新增, 小七車]",0.5
2,3,https://www.ptt.cc/bbs/car/M.1606830824.A.128....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01,car,cc一個月掛牌5319台國瑞產能這麼高丫?可以一天生產177台cc不知道它廠產能如何呢?像c...,"[月份, 臺灣汽車, 市場, 銷售, 報告, cc, 一個月, 掛牌, 5319, 國瑞, ...",0.5
3,4,https://www.ptt.cc/bbs/car/M.1606837278.A.BAB....,"[情報]日本到11月為止,各家新車累計銷量",2020-12-01,car,11月新車銷量 (車種銷量要等6號)https://response.jp/article...,"[日本, 各家, 新車, 累計, 銷量, 新車, 銷量, 銷量]",0.5
4,5,https://www.ptt.cc/bbs/car/M.1606843397.A.8A7....,Re:[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-02,car,Corolla Cross能賣破 5000 輛是真的挺強的，但畢竟還沒上市就壓了快6000張...,"[月份, 臺灣汽車, 市場, 銷售, 報告, Corolla, Cross, 5000, 輛...",0.5
...,...,...,...,...,...,...,...,...
2482,2483,https://www.ptt.cc/bbs/CarShop/M.1674817950.A....,[購車]2023HONDAHRVS+勁化版,2023-01-27,CarShop,徵求菜單：HONDA HRV S+ 勁化版車輛狀況：全新 和 全新領牌車都可 請各自報價跟註...,"[2023HONDAHRVS, 勁化版, HONDA, HRV, 勁化版, 領牌車, 註明,...",1.0
2484,2485,https://www.ptt.cc/bbs/CarShop/M.1675066184.A....,[購車]HondaCR-VVti-s版/S版,2023-01-30,CarShop,車輛狀況：2023年 全新車輛品牌：HONDA車款型式：CR-V Vti-s版及S版車輛顏色...,"[HondaCR, VVti, 2023, HONDA, CR, Vti, 版及, 北部, ...",0.5
2485,2486,https://www.ptt.cc/bbs/CarShop/M.1675077040.A....,[購車]HondaCRVVTI銀色,2023-01-30,CarShop,車輛狀況：全新車輛品牌：Honda車款型式：CRV Vti車輛顏色：銀色交易地區：不拘聯絡方...,"[HondaCRVVTI, 銀色, Honda, CRV, Vti, 銀色, 來信, 附上,...",1.0
2486,2487,https://www.ptt.cc/bbs/CarShop/M.1675084470.A....,[購車]5萬元以內代步車,2023-01-30,CarShop,車輛狀況：(二手)無待修或是待修項目不影響行車安全車輛品牌：不拘，以非進口車為優先車款型式：...,"[以內, 代步車, 無待修, 影響, 行車, 安全, 以非, 進口車, 優先, 門或, 大眾...",0.6


## 儲存結果

In [297]:
final_df.to_csv("../data/sentiment/mazda_clean_data.csv", encoding = 'utf-8',index = False)